Task1.Եթե որպես ֆունկցիայի պարամետր ստանում եք numpy զանգված (վեկտոր կամ մատրից), և այն չի համապատասխանում խնդրում նշված չափին (dimensions) կամ չափերի անհամաձայնություններ կան, ապա վերադարցրեք(return) **"DimensionError"** տեքստը

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
iris = load_iris()

data = iris['data']
feature_names = iris['feature_names']
target = iris['target']
target_names = iris['target_names']
def get_data_frame(data, feature_names, target, target_names):
  df = pd.DataFrame(data = data, columns = feature_names)
  df['target name'] = target_names[target]
  return df

df = get_data_frame(data, feature_names, target, target_names)
assert isinstance(df, pd.DataFrame)
assert df.shape == (150, 5)
assert np.all(df.loc[[0, 75, 120], 'target name'].values == target_names)

Task2.Գտեք յուրաքանչուր ծաղկատեսակի թերթիկի (պսակաթերթ և բաժակաթերթ)

1. միջին երկարությունը
2. միջին լայնությունը
3. նվազագույն երկարությունը
4. նվազագույն լայնությունը
5. առավելագույն երկարությունը
6. առավելագույն լայնությունը


In [ ]:
def get_stats(df):
    sepal_length = df.groupby('target name').agg({'sepal length (cm)' : ['mean', 'max', 'min']})
    sepal_length.columns = ["mean sepal length (cm)", "max sepal length (cm)", "min sepal length (cm)"]
    sepal_width = df.groupby('target name').agg({'sepal width (cm)' : ['mean', 'max', 'min']})
    sepal_width.columns = ["mean sepal width (cm)", "max sepal width (cm)", "min sepal width (cm)"]
    petal_length = df.groupby('target name').agg({'petal length (cm)' : ['mean', 'max', 'min']})
    petal_length.columns = ["mean petal length (cm)", "max petal length (cm)", "min petal length (cm)"]
    petal_width = df.groupby('target name').agg({'petal width (cm)' : ['mean', 'max', 'min']})
    petal_width.columns = ["mean petal width (cm)", "max petal width (cm)", "min petal width (cm)"]
    tmp = pd.concat([sepal_length, sepal_width, petal_length, petal_width], axis=1)
    return tmp

df_stats = get_stats(df)
assert df_stats.shape == (3, 12)
assert all(df_stats.index.to_list() == target_names)
assert np.allclose(df_stats.loc[:, "mean sepal length (cm)"].values,
            np.array([5.006, 5.936, 6.588]))
assert np.allclose(df_stats.loc[:, "min petal width (cm)"].values,
            np.array([0.1, 1.0, 1.4]))
assert np.allclose(df_stats.loc[:, "max petal length (cm)"].values,
            np.array([1.9, 5.1, 6.9]))

Task3. ``df`` աղյուսակից ջնջեք այն տողերը, որոնց 'sepal width'-ը փոքր է իր տեսակի միջինից (օգտվելով նախորդ խնդրի հաշվարկներից ``df_stats``)։ Փոփոխությունը կատարեք ``df``-ի վրա և ֆունկցիայով ոչինչ մի վերադարձրեք։

In [ ]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target name
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
def remove_small_flowers(df, df_stats):
  #Your code here
  meanS = df_stats["mean sepal width (cm)"][0]
  meanVer = df_stats["mean sepal width (cm)"][1]
  meanVirg = df_stats["mean sepal width (cm)"][2]

  result_set = np.vstack([(df["sepal width (cm)"][:50] < meanS).values, (df["sepal width (cm)"][50:100] < meanVer).values, (df["sepal width (cm)"][100:150] < meanVirg).values]).flatten()
  ind = df[result_set].index
  df.drop(index=ind, inplace = True)

In [ ]:
df.shape

(150, 5)

In [ ]:

# def remove_small_flowers(df, df_stats):
#     df = df.loc[((df.iloc[:, -1].values == df_stats.index[0]) & (df.iloc[:, 1].values >= df_stats.iloc[0, 3])) |
#         ((df.iloc[:, -1].values == df_stats.index[1]) & (df.iloc[:, 1].values >= df_stats.iloc[1, 3])) |
#          ((df.iloc[:, -1].values == df_stats.index[2]) & (df.iloc[:, 1].values >= df_stats.iloc[2, 3]))]
#     print(df.shape)
df_1 = df.loc[((df.iloc[:, -1].values == df_stats.index[0]) & (df.iloc[:, 1].values >= df_stats.iloc[0, 3])) |
        ((df.iloc[:, -1].values == df_stats.index[1]) & (df.iloc[:, 1].values >= df_stats.iloc[1, 3])) |
         ((df.iloc[:, -1].values == df_stats.index[2]) & (df.iloc[:, 1].values >= df_stats.iloc[2, 3]))]

remove_small_flowers(df, df_stats)

In [ ]:
df.shape, df_1.shape

((80, 5), (80, 5))

In [ ]:
(df == df_1).sum()

sepal length (cm)    80
sepal width (cm)     80
petal length (cm)    80
petal width (cm)     80
target name          80
dtype: int64

In [ ]:
# assert remove_small_flowers(df, df_stats) is None
print(df.shape)
assert df.shape == (80, 5)
assert (df[['petal length (cm)']].min() == 1.0)[0]
assert (df[['petal length (cm)']].max() == 6.7)[0]

(80, 5)
